In [100]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [101]:
# Example DataHunt CSV

df = pd.read_csv("testing-format/BETA_Probability-2020-05-20T0111-DataHunt.csv")
df.head(3)

,schema_namespace,schema_sha256,quiz_task_uuid,task_url,tua_uuid,article_batch_name,article_number,article_filename,article_sha256,article_text_length,...,answer_label,answer_content,answer_uuid,submitted_tua_uuid,answer_text,case_number,highlight_count,start_pos,end_pos,target_text
0,ProbabilitySpecialist_2019_08_05,65f41c5a11228e225f5a1f7fecdda76385de5b42626d8b...,33aeb7c3-b1f5-42a0-a94f-9a78d26ca939,https://pe.goodlylabs.org/project/BETA_Probabi...,7b3e1610-dd67-41ea-88f8-1f45dd5bcf9d,Demo2,1712,1712SSSArticle12.txt,38428e24cdfa0392f3f5e1efda3253ffab6d7ed3051b1d...,5385,...,T1.Q1.A1,Yes,526d6416-1d0f-40c6-8e9e-df8f44536bbf,0c9734e7-1ff8-4408-8db0-89e172d91ab5,Yes,1,1,3644,3677,it certainly seems to be the case
1,ProbabilitySpecialist_2019_08_05,65f41c5a11228e225f5a1f7fecdda76385de5b42626d8b...,33aeb7c3-b1f5-42a0-a94f-9a78d26ca939,https://pe.goodlylabs.org/project/BETA_Probabi...,7b3e1610-dd67-41ea-88f8-1f45dd5bcf9d,Demo2,1712,1712SSSArticle12.txt,38428e24cdfa0392f3f5e1efda3253ffab6d7ed3051b1d...,5385,...,T1.Q2.A3,Somewhat justified,7dff2434-b492-4089-8f8f-5da4704ce257,4bf28294-007a-4da4-85bb-89db89b5076e,Somewhat justified,0,0,0,0,NaN
2,ProbabilitySpecialist_2019_08_05,65f41c5a11228e225f5a1f7fecdda76385de5b42626d8b...,33aeb7c3-b1f5-42a0-a94f-9a78d26ca939,https://pe.goodlylabs.org/project/BETA_Probabi...,7b3e1610-dd67-41ea-88f8-1f45dd5bcf9d,Demo2,1712,1712SSSArticle12.txt,38428e24cdfa0392f3f5e1efda3253ffab6d7ed3051b1d...,5385,...,T1.Q5.A2,"Yes, implicitly",63440ba4-1f4b-4bf8-9d12-dd59c8429bfd,c51886b9-08d1-427f-a7aa-02df8e805c74,"Yes, implicitly",1,1,2574,2632,thought was likely the result of inappropriate...


In [102]:
df.columns

Index(['schema_namespace', 'schema_sha256', 'quiz_task_uuid', 'task_url',
       'tua_uuid', 'article_batch_name', 'article_number', 'article_filename',
       'article_sha256', 'article_text_length', 'destination',
       'task_redundancy', 'taskrun_count', 'quiz_taskrun_uuid',
       'contributor_uuid', 'created', 'finish_time', 'elapsed_seconds',
       'topic_name', 'question_label', 'question_text', 'answer_label',
       'answer_content', 'answer_uuid', 'submitted_tua_uuid', 'answer_text',
       'case_number', 'highlight_count', 'start_pos', 'end_pos',
       'target_text'],
      dtype='object')

In [103]:
# This is just a very simple one row dataset with the relevant columns, but the actual IAA output has more outputs
iaa = pd.read_csv("IAA-mock-output.csv")
iaa

,Article_num,Article_sha256,quiz_task_uuid,schema_namespace,num_answer_choices,question_Number,question_type,agreed_answer
0,1712,65f41c5a11228e225f5a1f7fecdda76385de5b42626d8b...,33aeb7c3-b1f5-42a0-a94f-9a78d26ca939,ProbabilitySpecialist_2019_08_05,3,1,Ordinal,1


Important columns:
- Article_num
- Article_sha256
- quiz_task_uuid
- schema_namespace
- num_answer_choices
- question_Number
- answer_uuid
- question_type
- agreed_answer

Function that takes in **article_sha256** (string), **quiz_task_uuid** (string), and **question_Number** (int), and returns the **corresponding consensus answer** from IAA.

The return value is a single int instead of the "T1.Q1.A1" format because I believe IAA processes and outputs just the number. 

(also let me know if there's a more efficient way to index the df)

In [104]:
def get_consensus(article_sha256, quiz_task_uuid, question_Number):
    answer_row = iaa.loc[(iaa["Article_sha256"]==article_sha256)
                  & (iaa["quiz_task_uuid"]==quiz_task_uuid)
                  & (iaa["question_Number"]==question_Number)]
    
    return answer_row["agreed_answer"].iloc[0]

In [105]:
get_consensus('65f41c5a11228e225f5a1f7fecdda76385de5b42626d8b7e4f6223081093b7f6',
             '33aeb7c3-b1f5-42a0-a94f-9a78d26ca939',
             1)

1

In [106]:
def get_consensus(article_sha256, quiz_task_uuid, question_Number):
    answer_row = iaa.loc[(iaa["Article_sha256"]==article_sha256)
                  & (iaa["quiz_task_uuid"]==quiz_task_uuid)
                  & (iaa["question_Number"]==question_Number)]
    
    return answer_row["agreed_answer"].iloc[0]

Function that takes in **article_sha256** (string), **quiz_task_uuid** (string), and **question_Number** (int), and returns a **tuple containing the question type and number of answer choices, will help with scoring questions**.

If the question_type and num_answer_choices works as expected, we shouldn't need to hardcode the question schema data anymore.

In [107]:
def get_question_meta(article_sha256, quiz_task_uuid, question_Number):
    answer_row = iaa.loc[(iaa["Article_sha256"]==article_sha256)
                  & (iaa["quiz_task_uuid"]==quiz_task_uuid)
                  & (iaa["question_Number"]==question_Number)]
    
    return (answer_row["question_type"].iloc[0], answer_row["num_answer_choices"].iloc[0])

In [108]:
get_question_meta('65f41c5a11228e225f5a1f7fecdda76385de5b42626d8b7e4f6223081093b7f6',
             '33aeb7c3-b1f5-42a0-a94f-9a78d26ca939',
             1)

('Ordinal', 3)